### Importing important packages

In [1101]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time 


### Downloading the data, defining the training and testing sets, and defining the classes

In [1102]:
transform = transforms.Compose(
    [transforms.ToTensor()])

training = torchvision.datasets.FashionMNIST(root='./',download=True,train=True,transform=transform)
testing = torchvision.datasets.FashionMNIST('./',download=True,train=False,transform=transform)

trainloader = torch.utils.data.DataLoader(training, batch_size=10,shuffle=True)
testloader = torch.utils.data.DataLoader(testing, batch_size=10,shuffle=True)



classes = ('T-shirt','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot')


### Defining the convolutional neural network, adding to an already existing neural net class

In [1103]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5,stride=2)
        self.batchN1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,12,5,stride=2)
        self.batchN2 = nn.BatchNorm1d(120)
        self.fc1 = nn.Linear(12*4*4, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 10)
        
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x,1,1)
        x = F.relu(x)
        x = self.batchN1(x)
        
        x = self.conv2(x)
        x = F.max_pool2d(x,1,1)
        x = F.relu(x)
        
        x = x.reshape(-1, 12*4*4)
        x = F.relu(self.fc1(x))
        x = self.batchN2(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
   
net = Net()


### Defining the device to perform the operations on, and also defining the loss and optimiser functions

In [1104]:
device = torch.device('cpu')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


### Training the algorithm on the training set

In [1105]:
epoch_num = 2
loss_list = []
t0 = time.time()
for epoch in range(epoch_num):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 50 == 0:
            loss_list.append(running_loss / 50)
            running_loss = 0.0
            #print(running_loss)
            


### Testing the network on the testing set

In [1106]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[1].item()
            class_total[label] += 1
            
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
t1 = time.time()
total = t1-t0


Accuracy of T-shirt : 81 %
Accuracy of Trouser : 85 %
Accuracy of Pullover : 82 %
Accuracy of Dress : 83 %
Accuracy of  Coat : 81 %
Accuracy of Sandal : 86 %
Accuracy of Shirt : 76 %
Accuracy of Sneaker : 86 %
Accuracy of   Bag : 87 %
Accuracy of Ankle Boot : 85 %


In [1107]:
print(f'Time taken to compute: {round(total, 2)}s')

Time taken to compute: 56.99s
